In [1]:
# 
# https://github.com/Pyomo/pyomo/blob/master/pyomo/contrib/pynumero/examples/feasibility.py
#

In [2]:
from pyomo.contrib.pynumero.interfaces import PyomoNLP
import matplotlib.pylab as plt
import pyomo.environ as aml
import numpy as np

In [3]:
from pyomo.version.info import version
print(version)

5.6.8


In [4]:
def create_basic_model():

    m = aml.ConcreteModel()
    m.x = aml.Var([1, 2, 3], domain=aml.Reals)
    for i in range(1, 4):
        m.x[i].value = i
    m.c1 = aml.Constraint(expr=m.x[1] ** 2 - m.x[2] - 1 == 0)
    m.c2 = aml.Constraint(expr=m.x[1] - m.x[3] - 0.5 == 0)
    m.d1 = aml.Constraint(expr=m.x[1] + m.x[2] <= 100.0)
    m.d2 = aml.Constraint(expr=m.x[2] + m.x[3] >= -100.0)
    m.d3 = aml.Constraint(expr=m.x[2] + m.x[3] + m.x[1] >= -500.0)
    m.x[2].setlb(0.0)
    m.x[3].setlb(0.0)
    m.x[2].setub(100.0)
    m.obj = aml.Objective(expr=m.x[2]**2)
    return m

In [5]:
model = create_basic_model()
solver = aml.SolverFactory('ipopt')
solver.solve(model, tee=True)

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        7
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 5, 'Number of variables': 3, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.11.1\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.07918190956115723}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [14]:
[model.x[i]() for i in range(1, 4)]

[1.0000228899543928, 4.578002230589221e-05, 0.5000228899543927]

In [6]:
# build nlp initialized at the solution
nlp = PyomoNLP(model)

In [9]:
print(nlp.variable_order())
x0 = nlp.x_init()
print(x0)

['x[1]', 'x[2]', 'x[3]']
[1.00002289e+00 4.57800223e-05 5.00022890e-01]


In [15]:
xl = nlp.xl(condensed=True)
xu = nlp.xu(condensed=True)
print(xl)
print(xu)

[0. 0.]
[100.]


In [16]:
# build expansion matrices
Pxl = nlp.expansion_matrix_xl()
Pxu = nlp.expansion_matrix_xu()

In [18]:
res_xl = Pxl.transpose() * x0 - xl
res_xu = xu - Pxu.transpose() * x0
print("Residuals lower bounds x-xl:", res_xl)
print("Residuals upper bounds xu-x:", res_xu)

Residuals lower bounds x-xl: [4.57800223e-05 5.00022890e-01]
Residuals upper bounds xu-x: [99.99995422]


In [19]:
print(nlp.constraint_order())
res_c = nlp.evaluate_c(x0)
print("Residuals equality constraints c(x):", res_c)
# evaluate residual of inequality constraints
d = nlp.evaluate_d(x0)

['c1', 'c2', 'd1', 'd2', 'd3']
Residuals equality constraints c(x): [4.10429690e-10 1.11022302e-16]


In [20]:
# compression matrices
Pdl = nlp.expansion_matrix_dl()
Pdu = nlp.expansion_matrix_du()

# vectors of finite lower and upper bounds
dl = nlp.dl(condensed=True)
du = nlp.du(condensed=True)


In [21]:
# lower and upper inequalities residual
res_dl = Pdl.transpose() * d - dl
res_du = du - Pdu.transpose() * d
print("Residuals lower bounds d-dl:", res_dl)
print("Residuals upper bounds du-d:", res_du)

Residuals lower bounds d-dl: [100.50006867 501.50009156]
Residuals upper bounds du-d: [98.99993133]


In [22]:
feasible = False
if np.all(res_xl >= 0) and np.all(res_xu >= 0) \
    and np.all(res_dl >= 0) and np.all(res_du >= 0) and \
    np.allclose(res_c, np.zeros(nlp.nc), atol=1e-5):
    feasible = True

print("Is x0 feasible:", feasible)

Is x0 feasible: True
